# GenerateScenarios
This notebook generates realistic compliance scenarios by leveraging AWS Bedrock's Retrieval-Augmented Generation (RAG) capabilities using the NIST control framework and specific organizational policies generated against that framework. It generates 1,000 (500 compliant, 500 non-compliant) complex, multi-policy scenarios that simulate real-world compliance situations (like employee onboarding, data access requests, or security incidents).

 - Generates Scenario Batches - Uses Bedrock's Converse API with RAG to create realistic compliance scenarios by alternating between compliant and non-compliant cases.  For each batch of scenarios, it retrieves policy context by querying AWS Bedrock Knowledge Base containing NIST controls and policies.
 - Structures Output - Forces JSON format output with specific schema including scenario ID, detailed description, compliance status, and violation reasons.
 - Multi-Policy Coverage - Each scenario incorporates multiple policies to create complex, realistic compliance evaluation situations.

```
Execution Flow:
└── main()
    ├── generate_compliance_scenarios()
    │   ├── retrieve_kb_context()
    │   │   └── bedrock_agent_runtime.retrieve()
    │   └── generate_scenario_batch()
    │       └── bedrock_runtime.converse()
    ├── save_scenarios_to_file()
    └── save_scenarios_to_s3()
```

In [23]:
import boto3
import json
import random
import re
import time
import threading
from botocore.exceptions import ClientError
from compliance_calculator import compliance_calculator, CALCULATOR_TOOL
from typing import List, Dict, Tuple, Callable, Any

BUCKET = '183023889407-us-east-1-compliance-rule-generator'
S3_PREFIX_POLICY_MARKDOWN_ALL = 'policies/markdown/all-policies-main/'
OUTPUT_PREFIX = 'scenarios/'  # Folder path for results
KNOWLEDGE_BASE_ID = 'T8EW10IU3Z'  # AWS Bedrock Knowledge Base containing NIST policies
# Available Bedrock model ARNs with performance notes
MODELS = {
    'premium': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/global.anthropic.claude-opus-4-5-20251101-v1:0', # not available
    'good': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/global.anthropic.claude-sonnet-4-5-20250929-v1:0', # times out
    'balanced': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.anthropic.claude-sonnet-4-20250514-v1:0',  # recommended
    'fast_cheap': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.anthropic.claude-haiku-4-5-20251001-v1:0',
    'aws_native_premier': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.amazon.nova-premier-v1:0',
    'aws_native_pro': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.amazon.nova-pro-v1:0'
}
MODEL_ARN = MODELS['premium']  # Default model selection
SCENARIOS_PER_BATCH = 4
NUM_BATCHES = 250
TOTAL_SCENARIOS = SCENARIOS_PER_BATCH * NUM_BATCHES
POLICIES_PER_SCENARIO = 4
MAX_RETRIES_ON_THROTTLE = 5

# JSON tool configuration for Bedrock Converse API
# Forces the model to return structured JSON with specific schema
TOOL_CONFIG = {
    "tools": [{
        "toolSpec": {
            "name": "scenario_json",
            "description": "Return compliance scenarios as JSON",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "scenarios": {  # Array of scenario objects
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "scenario-id": {"type": "string"},      # Format: scenario-id-1, scenario-id-2, etc.
                                    "scenario-detail": {"type": "string"},  # Detailed scenario description (200+ words)
                                    "is-compliant": {"type": "boolean"},     # True if compliant, False if non-compliant
                                    "non-compliant-reason": {"type": "string"}  # If non-compliant, why? Which policy(s) were violated?
                                },
                                "required": ["scenario-id", "scenario-detail", "is-compliant"]
                            }
                        }
                    },
                    "required": ["scenarios"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "compliance_calculator",
            "description": "Calculate and compare values with time, money, data, and percentage units",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "expression": {"type": "string", "description": "Expression like '800ms < 1s' or '4m > 3b'"}
                    },
                    "required": ["expression"]
                }
            }
        }
    }
              
             ]
}

# Initialize AWS Bedrock clients
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name='us-east-1')  # For knowledge base retrieval
bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-east-1')  # For model inference

In [24]:
def keep_alive():
    while True:
        time.sleep(300)  # 5 minutes
        print(f"Keep alive thread still running... {time.strftime('%Y-%m-%d %H:%M:%S')}")

In [25]:
def bedrock_call_with_retry(func: Callable[[], Any], max_retries: int = MAX_RETRIES_ON_THROTTLE, base_delay: int = 2) -> Any:
    for attempt in range(max_retries):
        try:
            return func()
        except ClientError as e:
            if e.response['Error']['Code'] == 'ThrottlingException':
                delay = base_delay * (2 ** attempt) # exponential backoff in the event of throttling
                print(f"Rate limit hit, waiting {delay}s...")
                time.sleep(delay)
            else:
                raise
    raise Exception("Max retries exceeded")

In [26]:
def generate_scenario_batch(
    policies: str,
    policy_ids: str,
    batch_num: int,
    model_arn: str,
    scenarios_per_batch: int=SCENARIOS_PER_BATCH
) -> List[Dict]:
    """
    Generate one batch of compliance scenarios using Bedrock Converse API.
    This function creates realistic compliance scenarios by alternating between
    compliant and non-compliant scenarios across batches. Uses the JSON tool
    to ensure structured output.
    Args:
        policies: Combined policy text from knowledge base
        batch_num: Current batch number (0-indexed)
        model_arn: AWS Bedrock model ARN or ID
        scenarios_per_batch: Number of scenarios per batch
    Returns:
        List of scenario dictionaries with keys: scenario-id, scenario-detail, is-compliant, non-compliant-reason
    """
    # Alternate between compliant (even batches) and non-compliant (odd batches) scenarios
    is_compliant = batch_num % 2 == 0
    
    # Calculate starting ID for consecutive numbering across all batches
    start_id = batch_num * scenarios_per_batch + 1
    
    # Construct detailed prompt with all policy context and specific requirements
    prompt = f"""Based on NIST compliance policies under **KNOWLEDGE BASE CONTEXT**, generate {scenarios_per_batch} realistic compliance scenario (scenario-details) 
    that are 
    {'compliant' if is_compliant else 'non-compliant.  Ensure that only one underlying policy is violated out of all the underlying policies for each scenario. Use no leading language like "however" or tone to indicate in any way that the scenarios is in any way invalid.  Try to trick the reader to believe it is valid.'}.
    
    Each scenario-detail must:
    - Use all policies from KNOWLEDGE BASE CONTEXT context below
    - Include specific business details (roles, systems, data, actions)
    - Be realistic and detailed (200+ words)
    - Have scenario-id format: scenario-id-{start_id}, scenario-id-{start_id+1}, etc.

    **Avoid generating scenarios based on cost-benefit principles or concentration percentages.**
    
    **Note that non-US citizens cannot obtain US security clearances.**

    **CRITICAL: For ANY numerical comparison involving timeframes, values, or thresholds:**
    - ALWAYS use the compliance_calculator tool to verify comparisons - do not do mental math
    - If a scenario meets or exceeds (performs better than) policy requirements, it is COMPLIANT.  For example, if a policy requires "within 24 hours" and scenario shows "within 18 hours", this is COMPLIANT (18 < 24).
    - If a scenario does not meet policy requirements it is NON-COMPLIANT.  For example, if a policy requires "at least quarterly (90 days)" and scenario shows "95 days", this is NON-COMPLIANT (95 > 90)

    
    Return as JSON array with fields: scenario-id, scenario-detail, is-compliant, non-compliant-reason
    {'leave non-compliant-reason empty' if is_compliant else 'For non-compliant-reason, provide an explanation of exactly why the scenario is non-compliant, including which policy(s) were violated.'}

    **Here is an example of scenario-detail:**
    SecureDefense Corporation, a defense contractor with 8,500+ employees supporting classified government projects, is deploying a new secure development environment
    for processing TOP SECRET/SCI information across 12 secure facilities. The Chief Security Officer implemented citizenship requirements per MA-5.3 ensuring all
    personnel performing maintenance on classified systems are verified U.S. citizens with current documentation including birth certificates, passports, and security
    clearance validation, with emergency maintenance procedures explicitly prohibiting non-citizen access under any circumstances. The organization deployed NIAP-approved
    protection profiles per SA-4.7 for all commercial security products including firewalls evaluated against Network Device Protection Profile v2.2, VPN solutions meeting
    IPsec Virtual Private Network (VPN) Client Protection Profile v1.0, and database management systems validated against Database Management System Protection Profile v4.0,
    with FIPS 140-2 Level 3 validated cryptographic modules for products without applicable NIAP profiles. The system architecture team implemented hierarchical protection
    per SA-8.12 with hypervisor components at highest trust level protecting against guest VMs, classified application containers at medium trust level isolated from
    development tools, and development interfaces at lowest trust level with comprehensive access controls preventing privilege escalation across trust boundaries. The
    procurement team maintained technology diversity per SC-29 across the secure development pipeline using Red Hat and SUSE Linux distributions (45% and 35% respectively),
    Dell and HPE hardware platforms, and Raytheon and General Dynamics security appliances to minimize supply chain concentration risks. Cost-benefit analyses per SA-8.25
    documented security investments of $4.2M annually justified by protecting classified intellectual property valued at $2.8B and avoiding potential contract termination
    penalties exceeding $150M for security violations.

    **KNOWLEDGE BASE CONTEXT**
    {policies}
    """
    # Extract model ID from ARN (Converse API requires model ID, not full ARN)
    model_id = model_arn.split('/')[-1] if '/' in model_arn else model_arn
    
    # Call Bedrock Converse API with JSON tool enforcement
    scenarios_data = {}
    scenarios = {}
    messages=[{"role": "user", "content": [{"text": prompt}]}]
    while True:

        response = bedrock_call_with_retry(
            lambda: bedrock_runtime.converse(
                        modelId=model_id,
                        messages=messages,
                        toolConfig=TOOL_CONFIG,  # Forces structured JSON output
                        inferenceConfig={"maxTokens": 4096, "temperature": 0.7}  # Allow creative but controlled generation, about 3,000 words max
            )
        )

        if response.get('stopReason') == 'tool_use':
            tool_results = []
            for content_block in response['output']['message']['content']:
                if 'toolUse' in content_block:
                    tool_name = content_block['toolUse']['name']
                    tool_use_id = content_block['toolUse']['toolUseId']

                    if tool_name == 'compliance_calculator':
                        expression = content_block['toolUse']['input']['expression']
                        result = compliance_calculator(expression)
                        # print("=" * 60)
                        # print(f"Compliance calculator expression: {expression}" )
                        # print(f"Compliance calculator result: {result}" )
                        # print("=" * 60)
                        tool_results.append({
                            "toolResult": {
                                "toolUseId": tool_use_id,
                                "content": [{"text": result}]
                            }
                        })
                    elif tool_name == 'scenario_json':
                        scenarios_data = content_block['toolUse']['input'] 
            
            if tool_results:
                messages.append({"role": "assistant", "content": response['output']['message']['content']})
                messages.append({"role": "user", "content": tool_results})
            else:
                break
        else:
            break
                               
    scenarios = scenarios_data.get('scenarios', [])

    # add policy ids used to each scenario
    for scenario in scenarios:
       scenario['scenario-detail'] += f"\n\nPolicies referenced: {policy_ids}"
    
    return scenarios

In [27]:
def retrieve_s3_policies(bucket: str = BUCKET, policies_per_scenario: int = POLICIES_PER_SCENARIO) -> Tuple[str, str]:
    """
    Retrieve random policy documents from S3 bucket.
    """
    s3 = boto3.client('s3')
    
    # List all policy files
    response = s3.list_objects_v2(
        Bucket=bucket,
        Prefix=S3_PREFIX_POLICY_MARKDOWN_ALL
    )
    
    policy_files = [obj['Key'] for obj in response.get('Contents', []) 
                   if obj['Key'].endswith('.md')]
    
    # Randomly select needed number of policies
    selected_files = random.sample(policy_files, min(policies_per_scenario, len(policy_files)))
    
    # Read selected policy contents
    policies = []
    policy_ids = []
    for file_key in selected_files:
        obj = s3.get_object(Bucket=bucket, Key=file_key)
        content = obj['Body'].read().decode('utf-8')
        policies.append(content)
        policy_ids.append("policy_" + re.search(r"POLICY: ([A-Z]+-[\d.]+)", content).group(1))
            
    print(f"Retrieved {len(policies)} random policies from S3")
    return '\n\n'.join(policies), ', '.join(policy_ids)


In [28]:
def generate_compliance_scenarios(
    knowledge_base_id: str = KNOWLEDGE_BASE_ID,
    model_arn: str = MODEL_ARN,
    scenarios_per_batch: int=SCENARIOS_PER_BATCH,
    policies_per_scenario: int=POLICIES_PER_SCENARIO,
    total_scenarios: int = TOTAL_SCENARIOS
) -> List[Dict]:
    """
    Main orchestrator function for generating compliance scenarios.
    
    This function coordinates the entire scenario generation process:
    1. Retrieves comprehensive policy context from knowledge base (once)
    2. Generates scenarios in batches to manage API limits and costs
    3. Alternates between compliant and non-compliant scenarios
    4. Implements rate limiting to avoid API throttling
    Args:
        knowledge_base_id: AWS Bedrock Knowledge Base ID
        model_arn: AWS Bedrock model ARN to use for generation
        scenarios_per_batch: Number of scenarios per batch
        policies_per_scenario: Number of NIST policies to use in creating a scenario
        total_scenarios: Total number of scenarios to generate
    Returns:
        List of all generated scenario dictionaries
    """
    
    # Generate scenarios in batches
    all_scenarios = []
    for batch_num in range(total_scenarios // scenarios_per_batch):
      # Retrieve all policy context once (reused across all batches)
        try:
            policies, policy_ids = retrieve_s3_policies(BUCKET, policies_per_scenario)
        except Exception as e:
            print(f"Error retrieving from KB: {e}")
            return []
        try:
            # Generate one batch of scenarios          
            scenarios = generate_scenario_batch(policies, policy_ids, batch_num, model_arn, scenarios_per_batch)
            all_scenarios.extend(scenarios)
            print(f"Batch {batch_num + 1}: Generated {len(scenarios)} scenarios")
        except Exception as e:
            print(f"Error generating scenario batch {batch_num + 1}: {e}")
            continue  # excplicit to go to next scenario, maybe this was a temporary glitch
        
        # Rate limiting: pause between batches to avoid API throttling
        time.sleep(2)
    
    return all_scenarios

In [29]:
def save_scenarios_to_file(scenarios: List[Dict], output_path: str):
    """
    Save generated scenarios to a JSON file with metadata.
    
    Creates a structured JSON file containing:
    - Summary statistics (total, compliant, non-compliant counts)
    - All generated scenarios
    """
    # Print scenarios to console for immediate review
    print(json.dumps(scenarios, indent=2))
    
    # Save to file with metadata and statistics
    with open(output_path, 'w') as f:
        json.dump({
            'total_scenarios': len(scenarios),
            'compliant_count': sum(1 for s in scenarios if s['is-compliant']),
            'non_compliant_count': sum(1 for s in scenarios if not s['is-compliant']),
            'scenarios': scenarios
        }, f, indent=2)

In [30]:
def save_scenarios_to_s3(scenarios: List[Dict], output_bucket: str = BUCKET, output_prefix: str = OUTPUT_PREFIX, object_name: str = "scenarios.json"):
    """
    Save generated scenarios to a S3.

    """
    s3 = boto3.client('s3')
    json_data = json.dumps({"scenarios": scenarios}, indent=2)
    s3.put_object(Bucket=output_bucket, Key=output_prefix+object_name, Body=json_data)


In [31]:
def main():
    # Example usage: Generate 4 scenarios in 2 batches of 2 each
    # Batch 0 (even): compliant scenarios with IDs scenario-id-1, scenario-id-2
    # Batch 1 (odd): non-compliant scenarios with IDs scenario-id-3, scenario-id-4
    
    # Start keep-alive thread
    # daemon=True parameter makes the thread automatically stop when the main program ends
    thread = threading.Thread(target=keep_alive, daemon=True)
    thread.start()
    
    scenarios = generate_compliance_scenarios(
        knowledge_base_id=KNOWLEDGE_BASE_ID,
        model_arn=MODELS['balanced'],
        scenarios_per_batch=SCENARIOS_PER_BATCH,
        policies_per_scenario=POLICIES_PER_SCENARIO,
        total_scenarios=TOTAL_SCENARIOS 
    )

    save_scenarios_to_file(scenarios, '/home/sagemaker-user/scenarios.json')
    save_scenarios_to_s3(scenarios, BUCKET, OUTPUT_PREFIX, "scenarios.json")


In [ ]:
main()

Retrieved 4 random policies from S3
